In [1]:
%run Functions.ipynb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('vacancies.csv')

In [3]:
# Обрабатываем колонку "Описание" и удаляем дубли и страны которых встречается меньше 10
df_dropna = (df[df['Описание'] != 'Описание не найдено']
             .assign(Описание=lambda x: x['Описание'].apply(clean_description_to_plain_text))
             .groupby('Страна')
             .filter(lambda x: len(x) >= 10)
             .drop_duplicates(subset='ID')
             .reset_index(drop=True))

# Сохраняем результат в CSV
df_dropna.to_csv("vacancies_cleaned.csv", index=False, encoding="utf-8-sig", quotechar='"')

In [4]:
df_clean = pd.read_csv('vacancies_cleaned.csv')
# Добавляем новые колонки с числовыми значениями зарплаты "на руки" и валютой
df_clean[['Зарплата на руки (числовое выражение)', 'Валюта']] = df_clean['Зарплата'].apply(
    lambda x: pd.Series(convert_salary_to_numeric(x))
)

# Применение функции которая считает зп 
df_clean['Зарплата, net., ₽'] = df_clean.apply(convert_to_rub, axis=1)

# Преобразуем колонку 'Дата публикации' в формат datetime
df_clean['Дата публикации'] = pd.to_datetime(df_clean['Дата публикации'])

# Создаем новую колонку с датой в формате yyyy-mm-dd
df_clean['Дата публикации (yyyy-mm-dd)'] = df_clean['Дата публикации'].dt.date

# Очищаем должности
df_clean['Кого ищут'] = df_clean['Кого ищут'].apply(clean_description_to_plain_text)

# Создание новой колонки 'Страна (full_name)' в df_clean
df_clean['Страна (full_name)'] = df_clean['Страна'].map(country_mapping)

# Создание новой колонки 'Тип занятости (full name)' в df_clean
df_clean['Тип занятости (full name)'] = df_clean['Тип занятости'].map(employment_type_mapping)

# Применение функции очистки текста и лемматизации, результат в новые колонки
df_clean['Описание (preProcess)'] = df_clean['Описание'].apply(preprocess_text)
df_clean['Кого ищут (preProcess)'] = df_clean['Кого ищут'].apply(preprocess_text)
df_clean.to_csv("vacancies_preprocess.csv", index=False, encoding="utf-8-sig", quotechar='"')

In [5]:
df_preprocess = pd.read_csv('vacancies_preprocess.csv')

In [6]:
# Применение функции к колонке и добавление новой колонки
df_preprocess['Вид дискриминации'] = df_preprocess['Описание (preProcess)'].apply(determine_discrimination_categories)
df_preprocess["Должность"] = df_preprocess["Кого ищут (preProcess)"].apply(categorize_job_vectorized)
df_preprocess['Дата публикации (yyyy-mm-dd)'] = pd.to_datetime(df_preprocess['Дата публикации (yyyy-mm-dd)'])
df_preprocess['Зарплата, net., ₽'] = df_preprocess['Зарплата, net., ₽'].apply(convert_to_integer)

In [7]:
df_preprocess['Вид дискриминации'].value_counts()

Вид дискриминации
Не найдено дискриминации                                                       979
Иные виды дискриминации                                                        153
Этническая дискриминация                                                        99
Возрастная дискриминация                                                        71
Возрастная дискриминация, Этническая дискриминация                              15
Иные виды дискриминации, Возрастная дискриминация                               15
Иные виды дискриминации, Этническая дискриминация                               14
Гендерная дискриминация                                                          5
Иные виды дискриминации, Возрастная дискриминация, Этническая дискриминация      4
Name: count, dtype: int64

In [ ]:
# итоговый датасет
df_final = df_preprocess[
    [
        'ID',
        'Кого ищут',
        'Описание',
        'Описание (preProcess)',
        'Компания',
        'Город',
        'Страна (full_name)',
        'Зарплата, net., ₽',
        'Дата публикации (yyyy-mm-dd)',
        'Тип занятости (full name)',
        'Вид дискриминации',
        'Должность'
    ]
].rename(columns={
    'Страна (full_name)': 'Страна',
    'Дата публикации (yyyy-mm-dd)': 'Дата публикации',
    'Тип занятости (full name)': 'Тип занятости'
})

# Замена "Зарплата не указана" на NaN
df_final['Зарплата, net., ₽'] = df_final['Зарплата, net., ₽'].replace('Зарплата не указана', pd.NA)

# Преобразование колонки в числовой тип
df_final['Зарплата, net., ₽'] = pd.to_numeric(df_final['Зарплата, net., ₽'], errors='coerce')

# Вычисление Z-score
df_final['z_score'] = stats.zscore(df_final['Зарплата, net., ₽'].dropna())

# Определение выбросов (значения с Z-score больше 3 или меньше -3)
outliers = df_final[(df_final['z_score'] > 3) | (df_final['z_score'] < -3)]

# Удаление выбросов
df_final = df_final[~df_final.index.isin(outliers.index)]

# Сброс индекса
df_final = df_final.reset_index(drop=True)

# Замена NaN на среднее значение по категории "Должность"
mean_salary_by_position = df_final.groupby('Должность')['Зарплата, net., ₽'].transform('mean')
df_final['Зарплата, net., ₽'].fillna(mean_salary_by_position, inplace=True)

# Если после группировки есть пропуски (например, все значения в группе NaN), можно заменить их на общее среднее
overall_mean_salary = df_final['Зарплата, net., ₽'].mean()
df_final['Зарплата, net., ₽'].fillna(overall_mean_salary, inplace=True)

# Округление до целого числа
df_final['Зарплата, net., ₽'] = df_final['Зарплата, net., ₽'].round().astype(int)

# Удаление временной колонки z_score
df_final = df_final.drop(columns=['z_score'])

# Разделяем лемматизированные слова запятыми
df_final['Описание (preProcess)'] = df_final['Описание (preProcess)'].apply(words_to_comma_separated)

# Сохранение DataFrame в CSV
df_final.to_csv("vacancies_final.csv", index=False, encoding="utf-8-sig", quotechar='"')